In [ ]:
import cvxopt as cvx
import numpy as np
import matplotlib.pyplot as plt

# from sklearn.datasets.samples_generator import make_blobs    # Para sklearn 0.22
from sklearn.datasets import make_blobs                        # Para nuevas versiones de sklearn
from sklearn.model_selection import train_test_split 

In [ ]:
class SVM(object):
    def fit_hard(self, X, y):
        """
        SVM de margen duro (Hard SVM)
        
        """
        n, d = X.shape   # n instancias, d atributos
        # Elementos para calcular alfa usando QP
        Xy = y.reshape(n,1)*X
        P = cvx.matrix( Xy.dot(Xy.T) )
        q = cvx.matrix( -1*np.ones(n) )
        A = cvx.matrix( y.reshape(1,n), tc='d' )
        b = cvx.matrix( 0.0 )
        G = cvx.matrix( -1*np.eye(n) )
        h = cvx.matrix( np.zeros(n) )
        
        # Calcular alfa
        solucion = cvx.solvers.qp(P, q, G, h, A, b)
        alfa = np.ravel(solucion['x'])
        alfa[alfa < 1e-5] = 0
        
        indice_sp = np.where(alfa>0)[0]   # Índices de los vectores de soporte
        n_sp = len(indice_sp)
        
        # Calcular w (pesos)
        self.w = np.zeros(d)
        for i in range(n_sp):
            self.w += alfa[indice_sp[i]] * y[indice_sp[i]] * X[indice_sp[i]]
        
        # Calcular b
        self.b = 0
        for i in range(n_sp):
            self.b += y[indice_sp[i]] - np.dot(self.w, X[indice_sp[i]] )
        self.b = self.b/n_sp
        
        return self.w, self.b
        
    def fit_soft(self, X, y, C):
        """
        SVM de margen blando (Soft SVM)
        
        """
        n, d = X.shape   # n instancias, d atributos
        # Elementos para calcular alfa usando QP
        Xy = y.reshape(n,1)*X
        P = cvx.matrix( Xy.dot(Xy.T) )
        q = cvx.matrix( -1*np.ones(n) )
        A = cvx.matrix( y.reshape(1,n), tc='d' )
        b = cvx.matrix( 0.0 )
        # Modificación de G para considerar C
        G = cvx.matrix( np.vstack(( -np.eye(n), np.eye(n) )) )
        # Modificación de h para considerar C
        h = cvx.matrix( np.hstack((np.zeros(n), C*np.ones(n))) )
        
        # Calcular alfa
        solucion = cvx.solvers.qp(P, q, G, h, A, b)
        alfa = np.ravel(solucion['x'])
        alfa[alfa < 1e-5] = 0

        # Calcular w (pesos): utiliza todos los valores de alfa
        self.w = np.zeros(d)
        for i in range(n):
            self.w += alfa[i] * y[i] * X[i]

        # Encontrar los valores de alfa correspondientes a los vectores de soporte
        # 0 < alfa < C     (epsilon se usa por los errores numéricos)
        indice_sp = np.where( (alfa > 0) & (alfa < (C-1e-10) ))[0]
        n_sp = len(indice_sp)

        # Calcular b: utiliza solo los valores de alfa donde 0 < alfa < C
        self.b = 0
        #print("np:", n_sp)
        for i in range(n_sp):
            self.b += y[indice_sp[i]] - np.dot(self.w, X[indice_sp[i]] )
        self.b = self.b/n_sp
        
        return self.w, self.b

In [ ]:
# Crear datos de ejemplo
X, y = make_blobs(n_samples=300, n_features=2, centers=2, random_state=5, cluster_std=0.5)
# Convertir los valores 0 de las clases negativas en valores -1, necesarios para el SVM
y[y==0] = -1

print("Tamaño de X:", X.shape)
print("Tamaño de y:", y.shape)

plt.scatter(X[:,0], X[:,1], c=y, cmap='winter')
plt.xlabel('$x_1$'); plt.ylabel('$x_2$');

In [ ]:
# ?train_test_split
# División de los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
# Aplicación del SVM implementado
model = SVM()
C = 0.1
w, b = model.fit_soft(X_train, y_train, C)

In [ ]:
def fline(x1, w, b, offset=0):
    """ Aplicación de una ecuación lineal a la entrada """
    x2 = (-w[0]*x1 - b + offset)/w[1]
    return x2

# Gráfico del resultado
plt.figure(figsize=(8,8))
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap='winter')

# Valores mínimos y máximos del eje de las abscisas 
x1min = np.min(X_train[:,0])
x1max = np.max(X_train[:,0])

# Gráfico de la frontera de separación
plt.plot([x1min, x1max], [fline(x1min,w,b,0), fline(x1max,w,b,0)], 'k')
# Gráfico de las rectas paralelas a la frontera de separación
plt.plot([x1min, x1max], [fline(x1min,w,b,1), fline(x1max,w,b,1)], 'r--')
plt.plot([x1min, x1max], [fline(x1min,w,b,-1), fline(x1max,w,b,-1)], 'r--');